# Co project p2

# data

In [1]:
V = [i for i in range(1, 9)]
A = [i for i in range(1, 19)]

In [2]:
#Routs
R= {1:[1,9,18], 2:[3,5,11,18], 3:[3,6,7,11,18],
    4:[3,6,12,17], 5:[1,9,15,17], 6:[5,8,12,14],
    7:[5,11], 8:[4,1,9], 9:[16,14,10,2],
    10:[16,13,2], 11:[13,2,3,5], 12:[14,10,2,3,6,7]}

In [3]:
print(R[1].index(18))

2


# model

In [4]:
import pyomo.environ as pyo 

In [5]:
model=pyo.ConcreteModel()

Sets and Params

In [6]:
model.Vset= pyo.Set(initialize= V)
model.Aset= pyo.Set(initialize= A)

In [7]:
Rr= [i for i in R]
model.Rset= pyo.Set(initialize= Rr)

In [8]:
exist= {(r,a): 1 if a in R[r] else 0 for r in model.Rset for a in model.Aset}

In [9]:
just_one= {(r_1,r_2,a): 1 if exist[(r_1,a)]!= exist[(r_2,a)] else 0 for r_1 in model.Rset for r_2 in model.Rset for a in model.Aset}

In [10]:
print(just_one[(3,4,7)])

1


In [11]:
order= {(r,a,b): 1 if exist[(r,a)] and exist[(r,b)] and R[r].index(a)>R[r].index(b) else 0 for r in model.Rset for a in model.Aset for b in model.Aset}

In [12]:
print(order[(3,11,11)])

0


In [13]:
dif_ord= {(r_1,r_2,a,b): 1 if order[(r_1,a,b)]!=order[(r_2,a,b)] else 0 for r_1 in model.Rset for r_2 in model.Rset for a in model.Aset for b in model.Aset}

Variables

In [14]:
model.x= pyo.Var(model.Aset, domain=pyo.Binary)

In [15]:
model.y= pyo.Var(model.Aset,model.Aset, domain=pyo.Binary)

Constraints

In [16]:
def const_exist(model,r):
    return sum(model.x[a]*exist[(r,a)] for a in model.Aset)>=1
model.const_exist= pyo.Constraint(model.Rset, rule= const_exist)

In [17]:
def const_y(model,a,b):
    if a==b:
        return pyo.Constraint.Skip
    else:
        return model.x[a]+model.x[b]<= 1+model.y[a,b]
model.const_y= pyo.Constraint(model.Aset, model.Aset, rule=const_y)

In [18]:
def const_y_1(model,a,b):
    if a==b:
        return pyo.Constraint.Skip
    else:
        return 2*model.y[a,b]<=model.x[a]+model.x[b]
model.const_y_1= pyo.Constraint(model.Aset, model.Aset, rule= const_y_1)

In [19]:
def dif(model,r_1,r_2):
    if r_1>=r_2:
        return pyo.Constraint.Skip
    else:
        return sum(model.x[a]* just_one[(r_1,r_2,a)] for a in model.Aset)+ sum(model.y[a,b]*dif_ord[(r_1,r_2,a,b)] for a in model.Aset for b in model.Aset if a!=b)>=1
model.const_dif= pyo.Constraint(model.Rset, model.Rset, rule= dif)

Objective

In [20]:
def obj(model):
    return sum(model.x[a] for a in model.Aset)
model.obj= pyo.Objective(rule=obj, sense= pyo.minimize)

Solve and result

In [21]:
opt = pyo.SolverFactory('glpk')
result=opt.solve(model,'glpk')

In [22]:
print(pyo.value(model.obj))

7.0


In [23]:
x_result={a: pyo.value(model.x[a]) for a in model.Aset}

In [24]:
print(x_result)

{1: 1.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 1.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 1.0, 14: 1.0, 15: 0.0, 16: 1.0, 17: 1.0, 18: 1.0}


In [25]:
model.display()

Model unknown

  Variables:
    x : Size=18, Index=Aset
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary
          5 :     0 :   1.0 :     1 : False : False : Binary
          6 :     0 :   0.0 :     1 : False : False : Binary
          7 :     0 :   0.0 :     1 : False : False : Binary
          8 :     0 :   0.0 :     1 : False : False : Binary
          9 :     0 :   0.0 :     1 : False : False : Binary
         10 :     0 :   0.0 :     1 : False : False : Binary
         11 :     0 :   0.0 :     1 : False : False : Binary
         12 :     0 :   0.0 :     1 : False : False : Binary
         13 :     0 :   1.0 :     1 : False : False : Binary
         14 :     0 :   1.0 :     1 : False : False : Binary
         15 :     0 :   0.0 :